# 1> basic cleanup:
    convertion to lower case
    removeral of white spaces
    removal of punctuations
    substitute of acronyms
    substitutions of contractions
    removal of stop words
    spelling corrections
    stemming and lemmetiosation
    discardments of non alphabetic words

In [26]:
import pandas as pd
import numpy as np
import string, re
import nltk
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
#pip install num2words
#from num2words import num2words
#!pip install pyspellchecker
#from spellchecker import SpellChecker
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy

In [27]:
NLP_data = pd.read_excel('NLP_Data.xlsx')

In [28]:
NLP_data

,Description of the Grievance,Grievance Category,Grievance SubCategory
0,concerns regarding laboratory tests being bill...,Billing/Financial Dispute,Provider Claim Issues
1,dassatifaction with provider,Quality Of Service,Not Satisfied With Provider Services
2,Dissatisafaction with delay in care.,Access And Availability,Pharmacy
3,Dissatisafaction with Dental provider way of c...,Quality Of Service,Not Satisfied With Provider Services
4,Dissatisfaction for not being informed he had ...,Billing/Financial Dispute,Balance Billing
...,...,...,...
520,Who: The member is requesting appeal Why: The...,Quality Of Care,Other
521,Why: Member states she was denied covid-19 bo...,Access And Availability,Other
522,NaN,Access And Availability,Other
523,NaN,Quality Of Care,Dental


# checking Duplicates and missing values 

In [ ]:
print(pd.Series({"NUmber of Observations with missing Value": len(NLP_data) - len(NLP_data.dropna()), "Number of duplicates observations": NLP_data.duplicated().sum()}).to_string())

###### droping all the blank data rows

In [3]:
NLP_data.dropna(inplace = True)
NLP_data.reset_index(drop = True, inplace = True)

NameError: name 'NLP_data' is not defined

###### encoding the grievances category coloumn to numberes in a new colomn"grievances category_ney"

In [6]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
NLP_data['Grievance Category_new'] = label_encoder.fit_transform(NLP_data['Grievance Category'])

In [7]:
NLP_data

,Description of the Grievance,Grievance Category,Grievance SubCategory,Grievance Category_new
0,concerns regarding laboratory tests being bill...,Billing/Financial Dispute,Provider Claim Issues,2
1,dassatifaction with provider,Quality Of Service,Not Satisfied With Provider Services,8
2,Dissatisafaction with delay in care.,Access And Availability,Pharmacy,0
3,Dissatisafaction with Dental provider way of c...,Quality Of Service,Not Satisfied With Provider Services,8
4,Dissatisfaction for not being informed he had ...,Billing/Financial Dispute,Balance Billing,2
...,...,...,...,...
494,Who: Member is appealing charge for no show of...,Quality Of Service,Other,8
495,Who: Member Why: the cost of the member's RX ...,Benefit Package,Other,1
496,Who: The mbr Why: The mbr is dissatisfied bec...,Quality Of Care,Pcp,7
497,Who: The member is requesting appeal Why: The...,Quality Of Care,Other,7


###### getting all the unique values for groevances category

In [8]:
NLP_data['Grievance Category'].unique()

array(['Billing/Financial Dispute', 'Quality Of Service',
       'Access And Availability', 'Quality Of Care',
       'Enrollment/Disenrollment', 'Benefit Package', 'Marketing',
       'Confidentiality/Privacy', 'Cms Issues'], dtype=object)

###### converting data to lower cases

In [9]:
data = []
for text in NLP_data['Description of the Grievance']:
    x = text.lower()
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

###### removing whitespaces

In [10]:
for text in NLP_data['Description_new']:
    x = text.strip()
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

###### removing punctuations

In [11]:
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "")
    return text.translate(str.maketrans("", "", punct_str))
for text in NLP_data['Description_new']:
    x = remove_punctuation(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

###### removing acronyms

In [12]:
# Dictionary of acronyms
acronyms_url = 'https://raw.githubusercontent.com/sugatagh/E-commerce-Text-Classification/main/JSON/english_acronyms.json'
acronyms_dict = pd.read_json(acronyms_url, typ = 'series')
# List of acronyms
acronyms_list = list(acronyms_dict.keys())
# Function to convert contractions in a text
def convert_acronyms(text):
    token = RegexpTokenizer('\s+', gaps = True)
    iterable = token.tokenize(text)
    words = []
    for word in iterable:
        if word in acronyms_list:
            words = words + acronyms_dict[word].split()
        else:
            words = words + word.split()
    
    text_converted = " ".join(words)
    return text_converted


In [13]:
for text in NLP_data['Description_new']:
    x = convert_acronyms(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

#### removing of contactions
contractions : a contraction is a shortened form of word or a phrase, obtained by dropping one or more letters

In [14]:
#contractions dictionary
contractions_url = 'https://raw.githubusercontent.com/sugatagh/E-commerce-Text-Classification/main/JSON/english_contractions.json'
contractions_dict = pd.read_json(contractions_url, typ = 'series')
#list of contraction
contractions_list = list(contractions_dict.keys())
def convert_contractions(text):
    token = RegexpTokenizer('\s+', gaps = True)
    iterable = token.tokenize(text)
    words = []
    for word in iterable:
        if word in contractions_list:
            words = words + contractions_dict[word].split()
        else:
            words = words + word.split()
    
    text_converted = " ".join(words)
    return text_converted
for text in NLP_data['Description_new']:
    x = convert_contractions(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

###### removing stopwords

In [15]:
# stopwords
nltk.download('stopwords')
stops = stopwords.words("english")
# additional stopwords
addstops = ["among", "onto", "shall", "thrice", "thus", "twice", "unto", "us", "would"]
allstops = stops + addstops
# Function to remove stopwords from a list of texts
def remove_stopwords(text):
    token = RegexpTokenizer('\s+', gaps = True)
    iterable = token.tokenize(text)
    return " ".join([word for word in iterable if word not in allstops])
for text in NLP_data['Description_new']:
    x = remove_stopwords(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

[nltk_data] Downloading package stopwords to C:\Users\Harshit
[nltk_data]     Anand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


###### spelling correction

###### stemming

stemming is process of reducing thw word to their root form or stem.

lemmatization provides a more suphistiacted approach by utilizing a corpus to match root forms of the words.

In [16]:
#stemming
stemmer = PorterStemmer()
def text_stemmer(text):
    token = RegexpTokenizer('\s+', gaps = True)
    iterable = token.tokenize(text)
    text_stem = " ".join([stemmer.stem(word) for word in iterable])
    return text_stem
for text in NLP_data['Description_new']:
    x = text_stemmer(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

In [17]:
spacy_lemmatizer = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])
def text_lemmatizer(text):
    text_spacy = " ".join([token.lemma_ for token in spacy_lemmatizer(text)])
    return text_spacy

In [18]:
for text in NLP_data['Description_new']:
    x = text_lemmatizer(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

###### discardment of non alphabetic words

In [19]:
def discard_non_alpha(text):
    token = RegexpTokenizer('\s+', gaps = True)
    iterable = token.tokenize(text)
    word_list_non_alpha = [word for word in iterable if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha
for text in NLP_data['Description_new']:
    x = discard_non_alpha(text)
    data.append(x)
NLP_data['Description_new'] = data
data.clear()

# vectorization

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
X_train_norm, y_train = NLP_data['Description_new'].tolist(), NLP_data['Grievance Category'].tolist()

In [24]:
X_val_norm, y_val = NLP_data['Description_new'].tolist(), NLP_data['Grievance Category'].tolist()

In [23]:
X_test_norm, y_test = NLP_data['Description_new'].tolist(), NLP_data['Grievance Category'].tolist()

In [25]:
TfidfVec = TfidfVectorizer(ngram_range = (1, 1))
X_train_tfidf = TfidfVec.fit_transform(X_train_norm)
X_val_tfidf = TfidfVec.transform(X_val_norm)
X_test_tfidf = TfidfVec.transform(X_test_norm)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [27]:
names = [
    "Logistic Regression",
    "KNN Classifier",
    "Decision Tree",
    "Linear SVM",
]

models = [
    LogisticRegression(max_iter = 1000),
    KNeighborsClassifier(n_neighbors = 149, n_jobs = -1),
    DecisionTreeClassifier(),
    svm.SVC(kernel = 'linear'),
]

In [55]:
def score(X_train, y_train, X_val, y_val, names = names, models = models):
    score_df, score_train, score_val = pd.DataFrame(), [], []
    x = time.time()
    for model in models:
        model.fit(X_train, y_train)
        y_train_pred, y_val_pred = model.predict(X_train), model.predict(X_val)
        score_train.append(accuracy_score(y_train, y_train_pred))
        score_val.append(accuracy_score(y_val, y_val_pred))
    
    score_df["Classifier"], score_df["Testing accuracy"] = names, score_train, score_val
    return score_df

In [56]:
import time, psutil, os
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [57]:
score(X_train_tfidf, y_train, X_val_tfidf, y_val, names = names, models = models)

NameError: name 'X_train_tfidf' is not defined